In [2]:
import logging
import sys
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings, StorageContext, load_index_from_storage
from llama_index.core.embeddings import resolve_embed_model
from llama_index.llms.ollama import Ollama

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
documents = SimpleDirectoryReader("data").load_data()

DEBUG:llama_index.core.readers.file.base:> [SimpleDirectoryReader] Total files added: 2
> [SimpleDirectoryReader] Total files added: 2
DEBUG:fsspec.local:open file: /Users/vishwajeetkumar/work/arm-template-generator/data/azure-storage-metadata.json
open file: /Users/vishwajeetkumar/work/arm-template-generator/data/azure-storage-metadata.json
DEBUG:fsspec.local:open file: /Users/vishwajeetkumar/work/arm-template-generator/data/azuredeploy-sotrage.json
open file: /Users/vishwajeetkumar/work/arm-template-generator/data/azuredeploy-sotrage.json


In [5]:
# bge embedding model
Settings.embed_model = resolve_embed_model("local:BAAI/bge-small-en-v1.5")
index = VectorStoreIndex.from_documents(
    documents,
)
index.storage_context.persist(persist_dir="./indexes") #persist indexes, might be helpful in future

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-small-en-v1.5/resolve/main/config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /BAAI/bge-small-en-v1.5/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-small-en-v1.5/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /BAAI/bge-small-en-v1.5/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: {
  "$schema": "https://aka.ms/azure-quickstart...
> Adding chunk: {
  "$schema": "https://aka.ms/azure-quickstart...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: {
    "$schema": "https://schema.management.azu...
> Adding chunk: {
    "$schema": "https://schema.management.azu...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: If false, then all requests, including shared a...
> Adding chunk: If false, then all requests, including sh

In [6]:
query_engine = index.as_query_engine()
response = query_engine.query("Give me a JSON arm template to deploy a storage account. It should contain default values")
print(response)

DEBUG:llama_index.core.indices.utils:> Top 2 nodes:
> [Node 8ad3e3ac-aaa4-47e7-8464-8d518fb3f664] [Similarity score:             0.803531] {
  "$schema": "https://aka.ms/azure-quickstart-templates-metadata-schema#",
  "type": "Module",
...
> [Node 19cfae5c-7553-478d-8772-0ab58faa2e34] [Similarity score:             0.784214] {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentTemplate.json#...
> Top 2 nodes:
> [Node 8ad3e3ac-aaa4-47e7-8464-8d518fb3f664] [Similarity score:             0.803531] {
  "$schema": "https://aka.ms/azure-quickstart-templates-metadata-schema#",
  "type": "Module",
...
> [Node 19cfae5c-7553-478d-8772-0ab58faa2e34] [Similarity score:             0.784214] {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentTemplate.json#...
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations caf

In [13]:
response_str = str(response)
parsed_json = json.loads(response_str)
pretty_json = json.dumps(parsed_json, indent=4)
print(pretty_json)

{
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentTemplate.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "location": {
            "type": "string",
            "defaultValue": "[resourceGroup().location]"
        },
        "storageAccountName": {
            "type": "string",
            "defaultValue": "[concat('storage', uniqueString(resourceGroup().id))]"
        },
        "skuName": {
            "type": "string",
            "defaultValue": "Standard_LRS"
        },
        "kind": {
            "type": "string",
            "defaultValue": "StorageV2"
        },
        "accessTier": {
            "type": "string",
            "defaultValue": "Hot"
        },
        "minimumTlsVersion": {
            "type": "string",
            "defaultValue": "TLS1_2"
        },
        "supportsHttpsTrafficOnly": {
            "type": "bool",
            "defaultValue": true
        },
        "allowBlobPublicAccess": {
            "